In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
img_path = "/home/nipun/Documents/Uni_Malta/Datasets/gi4e_database/images"
label_dir = "/home/nipun/Documents/Uni_Malta/Datasets/gi4e_database/labels"
saved_location = "/home/nipun/Documents/Uni_Malta/Datasets/gi4e_database/Visualization"


if not os.path.exists(saved_location):
    os.makedirs(saved_location)

In [ ]:
# Data Visualization

for text_path in os.scandir(label_dir):
    
    with open(text_path.path,"r") as d:
        
        info = d.readlines()
        
        
    for line in info:
        data = line.strip().split("\t")
        
        img_name = data[0]

        bbox_array = np.array_split(np.array(data[1:]),6)
        
        
        img = cv2.imread(os.path.join(img_path, img_name))
        
        
        for bbox in bbox_array:
            cv2.circle(img,(int(float(bbox[0])),int(float(bbox[1]))),1,(255,0,0),-1)
            
        cv2.imwrite(os.path.join(saved_location,img_name),img)
        
    # break
        

# Eye Cropping

In [ ]:
import mediapipe 
import pandas as pd


In [ ]:
text_file = "/home/nipun/Documents/Uni_Malta/Datasets/gi4e_database/labels/067_image_labels.txt"
img_path = "/home/nipun/Documents/Uni_Malta/Datasets/gi4e_database/images/067_08.png"

In [ ]:
mp_face_mesh = mediapipe.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

In [ ]:
img_base= cv2.imread(img_path)
results = face_mesh.process(img_base)

landmarks = results.multi_face_landmarks[0]

In [ ]:

if results.multi_face_landmarks is not None:
    print("Mediapipe Detected the Face")
else:
    print("MediaPipe Couln't be able to detect the face")

In [ ]:
img_1 = img_base.copy()
xs = [];ys = [];zs = [];
for landmark in landmarks.landmark:
    x = landmark.x
    y = landmark.y
    # z = landmark.z
    
    relative_x = int(x * img_base.shape[1])
    relative_y = int(y * img_base.shape[0])
    
    cv2.circle(img_1,(relative_x,relative_y),radius = 2,color=(0,255,0),thickness=-1)

plt.figure(figsize=(15,15))
plt.imshow(img_1[:,:,::-1])

In [ ]:
# Convert MediaPipe Array to Numpy Array:
shape_arr = []


for landmark in landmarks.landmark:
    x = landmark.x
    y = landmark.y
    
    relative_x = int(x * img_base.shape[1])
    relative_y = int(y * img_base.shape[0])
    
    
    shape_arr.append([relative_x, relative_y])


shape_arr = np.array(shape_arr) #( 468 ,2)

In [ ]:
img_landmark = img_base.copy()

lEyeTopLeftCorner = shape_arr[70]
rEyeTopLeftCorner = shape_arr[285]





lEyeBottomRight = shape_arr[133] 

rEyeBottomRight  = shape_arr[263]

In [ ]:
cv2.circle(img_landmark,lEyeTopLeftCorner,2,(0,0,255),-1)
cv2.circle(img_landmark,rEyeTopLeftCorner,2,(0,0,255),-1)

cv2.circle(img_landmark,lEyeBottomRight,2,(0,0,255),-1)
cv2.circle(img_landmark,rEyeBottomRight,2,(0,0,255),-1)

fig = plt.figure(figsize=(15,15))
plt.imshow(img_landmark[:,:,::-1])
plt.show()

In [ ]:
def cropped_image(img, shape_array, padded_amt=15):
    """Cropped eye region 

    Args:
        img (__numpy__): _Original Image_
        shape_array (_numpy_): _FaceLandMark locations_
        padded_amt (int, optional): _padding size_. Defaults to 20.
        
    """

    Leye = {"top_left": shape_array[70], "bottom_right": shape_array[133]}

    Reye = {"top_left": shape_array[285] ,
            "bottom_right": shape_array[446]}

    left_eye = img[Leye["top_left"][1]:Leye["bottom_right"][1] +
                   padded_amt, Leye["top_left"][0]:Leye["bottom_right"][0]]

    right_eye = img[Reye["top_left"][1]:Reye["bottom_right"][1] +
                    padded_amt, Reye["top_left"][0]:Reye["bottom_right"][0]]
    
    
    return left_eye, right_eye,Leye,Reye

In [ ]:
left_eye, right_eye,Leye,Reye = cropped_image(img_base,shape_arr)

In [ ]:
plt.imshow(left_eye[:,:,::-1])

In [ ]:
plt.imshow(right_eye[:,:,::-1])

In [ ]:
with open(text_file,"r") as data:
    
    info = data.readlines()

In [ ]:
bbox = info[0]

In [ ]:
print(bbox)

coor = [490.63,338.28,471.22	,338.41	,452.86,	341.93,	401.95	,342.45,	383.33,	339.84,	362.63,	338.41]

In [ ]:
left_center = coor[8:10]

print(left_center)

In [ ]:
rcoord = [490.63,338.28,471.22,338.41,452.86,341.93]

lcoord = [401.95,342.45,383.33,339.84,362.63,338.41]

In [ ]:
def getCenterofEye(coord,leye,reye):
    
    left_center = coord[8:10]
    right_center = coord[2:4]
    
    
    left_center = [float(left_center[0])-leye["top_left"][0],float(left_center[1])-leye["top_left"][1]]
    
    right_center = [float(right_center[0])-reye["top_left"][0],float(right_center[1])-reye["top_left"][1]]
    
    
    return left_center,right_center
    
    
        

In [ ]:
print(Leye["top_left"][0])

print(Reye)

In [ ]:
cv2.circle(left_eye,(int(lcoord[2])-350,int(lcoord[3])-322),2,(0,0,255),-1)
cv2.circle(right_eye,(int(rcoord[2])-444,int(rcoord[3])-335),2,(0,0,255),-1)

In [ ]:
plt.imshow(left_eye[:,:,::-1])